In [ ]:
import torch
import torchvision 
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))])

transform_flip = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=1.0),
     transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))])

batch_size = 64
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset_flip = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_flip)

trainloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([trainset, trainset_flip]), batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.norm2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.2)

        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.norm3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.norm4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.3)

        self.conv5 = nn.Conv2d(64, 64, 3, padding=1)
        self.norm5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(64, 64, 3, padding=1)
        self.norm6 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.drop3 = nn.Dropout(0.4)

        self.fc1 = nn.Linear(64* 4* 4, 128)
        self.norm7 = nn.BatchNorm1d(128)
        self.drop4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.norm1(F.relu(self.conv1(x)))
        x = self.norm2(F.relu(self.conv2(x)))
        x = self.drop1(self.pool1(x))

        x = self.norm3(F.relu(self.conv3(x)))
        x = self.norm4(F.relu(self.conv4(x)))
        x = self.drop2(self.pool2(x))

        x = self.norm5(F.relu(self.conv5(x)))
        x = self.norm6(F.relu(self.conv6(x)))
        x = self.drop3(self.pool3(x))
        
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = self.drop4(self.norm7(x))
        x = F.softmax(self.fc2(x))
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 20
log_interval = 16
accuracy_hist = []
loss_hist = []


for epoch in range(epochs):
    accuracy_avr = 0
    loss_avr = 0
    for batch_idx, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        net_out = net(inputs)
        accuracy = (torch.argmax(net_out, 1) == labels).sum()/trainloader.batch_size
 
        loss = criterion(net_out, labels)
        loss.backward()
        optimizer.step()

        accuracy_avr += accuracy
        loss_avr += loss.data

        if batch_idx % log_interval == 0 and batch_idx != 0:
          accuracy_hist.append(accuracy_avr/log_interval)
          loss_hist.append(loss_avr/log_interval)

          if batch_idx % (log_interval) == 0:
            print('epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAccuracy: {:.6f} '.format(
                epoch, batch_idx * len(inputs), len(trainloader.dataset),
                        100. * batch_idx / len(trainloader), loss_avr/log_interval, accuracy_avr/log_interval))
          accuracy_avr = 0
          loss_avr = 0

print('Finished Training')

In [ ]:
fig = plt.figure(figsize = (20, 10))
plt.subplot(2, 1, 1)
plt.plot(accuracy_hist)
plt.subplot(2, 1, 2)
plt.plot(loss_hist)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

In [ ]:
net.eval() 

dummy_input = torch.randn(1, 3, 32, 32, requires_grad=True)  
  
torch.onnx.export(net, dummy_input, "network.onnx",  
      export_params=True, opset_version=10, do_constant_folding=True,
      input_names = ['modelInput'], output_names = ['modelOutput'], verbose=True) 

In [ ]:
test_size = 200
input = np.zeros([test_size, 32, 32, 3])
output = np.zeros([test_size, 10])
for i in range(test_size):
  for c, chanal in enumerate(np.array(testloader.dataset[i][0])):
   for l, line in enumerate(chanal):
     for v, item in enumerate(line):
        input[i][l][v][c] = item
  output[i][testloader.dataset[i][1]] = 1.0

np.save('input', input)
np.save('output', output)